In [ ]:
# create and evealuate a stacking ensemble of the best three models and their optimized hyperparameters

import numpy as np
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load datasets
main_data = pd.read_csv("./data/train.csv")
unique_m = pd.read_csv("./data/unique_m.csv")

# Remove 'critical_temp' from unique_m to avoid duplication
unique_m = unique_m.drop(columns=["critical_temp"], errors='ignore')

# Merge datasets assuming rows align (index-based merge)
merged_data = pd.concat([main_data, unique_m], axis=1)

# Feature Engineering: Physics-Based Ratio, Thermal Conductivity Transformation, Log transformation
merged_data["mass_density_ratio"] = merged_data["wtd_mean_atomic_mass"] / (merged_data["wtd_mean_Density"] + 1e-9)
merged_data["affinity_valence_ratio"] = merged_data["wtd_mean_ElectronAffinity"] / (merged_data["wtd_mean_Valence"] + 1e-9)
merged_data["log_thermal_conductivity"] = np.log1p(merged_data["range_ThermalConductivity"])

# Define target and features
target = "critical_temp"
features = ['mean_atomic_mass', 'wtd_mean_atomic_mass', 'gmean_atomic_mass',
       'entropy_atomic_mass', 'wtd_entropy_atomic_mass', 'range_atomic_mass',
       'wtd_range_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie',
       'wtd_mean_fie', 'wtd_entropy_fie', 'range_fie', 'wtd_range_fie',
       'wtd_std_fie', 'mean_atomic_radius', 'wtd_mean_atomic_radius',
       'gmean_atomic_radius', 'range_atomic_radius', 'wtd_range_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density', 'entropy_Density',
       'wtd_entropy_Density', 'range_Density', 'wtd_range_Density',
       'wtd_std_Density', 'mean_ElectronAffinity', 'wtd_mean_ElectronAffinity',
       'gmean_ElectronAffinity', 'wtd_gmean_ElectronAffinity',
       'entropy_ElectronAffinity', 'wtd_entropy_ElectronAffinity',
       'range_ElectronAffinity', 'wtd_range_ElectronAffinity',
       'wtd_std_ElectronAffinity', 'mean_FusionHeat', 'wtd_mean_FusionHeat',
       'gmean_FusionHeat', 'entropy_FusionHeat', 'wtd_entropy_FusionHeat',
       'range_FusionHeat', 'wtd_range_FusionHeat', 'wtd_std_FusionHeat',
       'mean_ThermalConductivity', 'wtd_mean_ThermalConductivity',
       'gmean_ThermalConductivity', 'wtd_gmean_ThermalConductivity',
       'entropy_ThermalConductivity', 'wtd_entropy_ThermalConductivity',
       'range_ThermalConductivity', 'wtd_range_ThermalConductivity',
       'mean_Valence', 'wtd_mean_Valence', 'range_Valence',
       'wtd_range_Valence', 'wtd_std_Valence', 'H', 'B', 'C', 'O', 'F', 'Na',
       'Mg', 'Al', 'Cl', 'K', 'Ca', 'V', 'Cr', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
       'As', 'Se', 'Sr', 'Y', 'Nb', 'Sn', 'I', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
       'Sm', 'Eu', 'Gd', 'Tb', 'Yb', 'Hg', 'Tl', 'Pb', 'Bi',
       'mass_density_ratio', 'affinity_valence_ratio',
       'log_thermal_conductivity']
X = merged_data[features]
y = merged_data[target]


# Define base models with their optimized parameters
base_estimators = [
    ('xgb', XGBRegressor(
         n_estimators=374, max_depth=16, learning_rate=0.02, 
         min_child_weight=1, colsample_bytree=0.5, random_state=42, 
         objective='reg:squarederror')),
    ('lgb', LGBMRegressor(
         n_estimators=496, max_depth=15, learning_rate=0.0579, 
         subsample=0.6619, colsample_bytree=0.7512, num_leaves=148, verbose=-1,
         random_state=42)),
    ('cat', CatBoostRegressor(
         iterations=998, learning_rate=0.0962, depth=9, 
         l2_leaf_reg=4.1926, loss_function='RMSE', random_seed=42, verbose=0))
]

# Define a meta-model (here, we use a simple linear regression)
meta_model = LinearRegression()

# Create the stacking ensemble
stacking_model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=meta_model,
    cv=5  # use 5-fold CV to generate out-of-fold predictions
)




# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 90/10 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42 + i)
    
    # Fit the model on the training set
    stacking_model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = stacking_model.predict(X_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")


Results:

Run 1: RMSE = 8.2961, R² = 0.9400
Run 2: RMSE = 8.8419, R² = 0.9317
Run 3: RMSE = 8.6560, R² = 0.9349
Run 4: RMSE = 8.4912, R² = 0.9378
Run 5: RMSE = 9.0950, R² = 0.9282
Run 6: RMSE = 8.5501, R² = 0.9351
Run 7: RMSE = 8.9674, R² = 0.9318
Run 8: RMSE = 8.5989, R² = 0.9362
Run 9: RMSE = 8.2002, R² = 0.9430
Run 10: RMSE = 8.2798, R² = 0.9421
Run 11: RMSE = 8.8596, R² = 0.9351
Run 12: RMSE = 9.7608, R² = 0.9216
Run 13: RMSE = 8.6356, R² = 0.9365
Run 14: RMSE = 8.1875, R² = 0.9424
Run 15: RMSE = 8.5835, R² = 0.9390
Run 16: RMSE = 9.1220, R² = 0.9277
Run 17: RMSE = 8.6026, R² = 0.9370
Run 18: RMSE = 8.4384, R² = 0.9393
Run 19: RMSE = 9.2660, R² = 0.9251
Run 20: RMSE = 8.5692, R² = 0.9360
Run 21: RMSE = 8.9383, R² = 0.9322
Run 22: RMSE = 9.6753, R² = 0.9204
Run 23: RMSE = 8.5737, R² = 0.9379
Run 24: RMSE = 9.3588, R² = 0.9265
Run 25: RMSE = 8.4878, R² = 0.9385

Average RMSE over 25 runs: 8.7614

Average R² over 25 runs: 0.9342